In [1]:
import torch
import torchvision.models as models
import tarfile

resnet18 = models.resnet18(pretrained=True)
input_shape = [1, 3, 224, 224]
trace = torch.jit.trace(resnet18.float().eval(), torch.zeros(input_shape).float())
trace.save("model.pth")

with tarfile.open("model.tar.gz", "w:gz") as f:
    f.add("model.pth")
    f.add("resnet18.py")

In [2]:
trace

ResNet(
  original_name=ResNet
  (conv1): Conv2d(original_name=Conv2d)
  (bn1): BatchNorm2d(original_name=BatchNorm2d)
  (relu): ReLU(original_name=ReLU)
  (maxpool): MaxPool2d(original_name=MaxPool2d)
  (layer1): Sequential(
    original_name=Sequential
    (0): BasicBlock(
      original_name=BasicBlock
      (conv1): Conv2d(original_name=Conv2d)
      (bn1): BatchNorm2d(original_name=BatchNorm2d)
      (relu): ReLU(original_name=ReLU)
      (conv2): Conv2d(original_name=Conv2d)
      (bn2): BatchNorm2d(original_name=BatchNorm2d)
    )
    (1): BasicBlock(
      original_name=BasicBlock
      (conv1): Conv2d(original_name=Conv2d)
      (bn1): BatchNorm2d(original_name=BatchNorm2d)
      (relu): ReLU(original_name=ReLU)
      (conv2): Conv2d(original_name=Conv2d)
      (bn2): BatchNorm2d(original_name=BatchNorm2d)
    )
  )
  (layer2): Sequential(
    original_name=Sequential
    (0): BasicBlock(
      original_name=BasicBlock
      (conv1): Conv2d(original_name=Conv2d)
      (bn1): B

In [3]:
import boto3
import sagemaker
import time
from sagemaker.utils import name_from_base

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()

compilation_job_name = name_from_base("TorchVision-ResNet18-Neo-Inf1")

model_key = "{}/model/model.tar.gz".format(compilation_job_name)
model_path = "s3://{}/{}".format(bucket, model_key)
boto3.resource("s3").Bucket(bucket).upload_file("model.tar.gz", model_key)
print("Uploaded model to s3:")
print(model_path)

sm_client = boto3.client("sagemaker")
compiled_model_path = "s3://{}/{}/output".format(bucket, compilation_job_name)
print("Output path for compiled model:")
print(compiled_model_path)

Uploaded model to s3:
s3://sagemaker-us-west-2-688520471316/TorchVision-ResNet18-Neo-Inf1-2021-05-28-22-33-18-829/model/model.tar.gz
Output path for compiled model:
s3://sagemaker-us-west-2-688520471316/TorchVision-ResNet18-Neo-Inf1-2021-05-28-22-33-18-829/output


In [4]:
# Define a PyTorch Model

from sagemaker.pytorch.model import PyTorchModel

pytorch_model = PyTorchModel(
    model_data=model_path,
    role=role,
    entry_point="resnet18.py",
    framework_version="1.5.1",
    py_version="py3",
)

In [6]:
print(compilation_job_name)

TorchVision-ResNet18-Neo-Inf1-2021-05-28-22-33-18-829


In [13]:
# Create Compilation Job
import json

sm_cli = boto3.client('sagemaker')

input_config = {
    "S3Uri": model_path,
    "DataInputConfig": json.dumps({"input0": [1, 3, 224, 224]}),
    "Framework" : "PYTORCH",
    #"FrameworkVersion": "1.5.1",
}

output_config = {
    "S3OutputLocation" : compiled_model_path,
    "TargetDevice": "ml_inf1",
    'CompilerOptions': json.dumps({"num-neuroncores":2})
}


sm_cli.create_compilation_job(
    CompilationJobName = compilation_job_name,
    RoleArn = role,
    InputConfig=input_config,
    OutputConfig=output_config,
    StoppingCondition={
        'MaxRuntimeInSeconds': 1200,
        'MaxWaitTimeInSeconds': 1200
    },
)

{'CompilationJobArn': 'arn:aws:sagemaker:us-west-2:688520471316:compilation-job/TorchVision-ResNet18-Neo-Inf1-2021-05-28-22-33-18-829',
 'ResponseMetadata': {'RequestId': '5223e070-a348-4ef1-9475-0548dca67584',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5223e070-a348-4ef1-9475-0548dca67584',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '134',
   'date': 'Fri, 28 May 2021 22:54:12 GMT'},
  'RetryAttempts': 0}}

In [17]:
sm_cli.describe_compilation_job(
    CompilationJobName=compilation_job_name)

{'CompilationJobName': 'TorchVision-ResNet18-Neo-Inf1-2021-05-28-22-33-18-829',
 'CompilationJobArn': 'arn:aws:sagemaker:us-west-2:688520471316:compilation-job/TorchVision-ResNet18-Neo-Inf1-2021-05-28-22-33-18-829',
 'CompilationJobStatus': 'FAILED',
 'CompilationStartTime': datetime.datetime(2021, 5, 28, 22, 54, 13, tzinfo=tzlocal()),
 'CompilationEndTime': datetime.datetime(2021, 5, 28, 23, 10, 6, tzinfo=tzlocal()),
 'StoppingCondition': {'MaxRuntimeInSeconds': 1200},
 'CreationTime': datetime.datetime(2021, 5, 28, 22, 54, 12, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2021, 5, 28, 23, 10, 6, tzinfo=tzlocal()),
 'FailureReason': "ClientError: CompilationError: Unable to compile model for ml_inf1:', 'No operations were successfully partitioned and compiled to neuron for this model - aborting trace!')",
 'RoleArn': 'arn:aws:iam::688520471316:role/hongshan-sagemaker-experiment',
 'InputConfig': {'S3Uri': 's3://sagemaker-us-west-2-688520471316/TorchVision-ResNet18-Neo-Inf1

In [ ]:

response = client.create_compilation_job(
    CompilationJobName='string',
    RoleArn='string',
    InputConfig={
        'S3Uri': 'string',
        'DataInputConfig': 'string',
        'Framework': 'TENSORFLOW'|'KERAS'|'MXNET'|'ONNX'|'PYTORCH'|'XGBOOST'|'TFLITE'|'DARKNET'|'SKLEARN',
        'FrameworkVersion': 'string'
    },
    OutputConfig={
        'S3OutputLocation': 'string',
        'TargetDevice': 'lambda'|'ml_m4'|'ml_m5'|'ml_c4'|'ml_c5'|'ml_p2'|'ml_p3'|'ml_g4dn'|'ml_inf1'|'ml_eia2'|'jetson_tx1'|'jetson_tx2'|'jetson_nano'|'jetson_xavier'|'rasp3b'|'imx8qm'|'deeplens'|'rk3399'|'rk3288'|'aisage'|'sbe_c'|'qcs605'|'qcs603'|'sitara_am57x'|'amba_cv22'|'x86_win32'|'x86_win64'|'coreml'|'jacinto_tda4vm',
        'TargetPlatform': {
            'Os': 'ANDROID'|'LINUX',
            'Arch': 'X86_64'|'X86'|'ARM64'|'ARM_EABI'|'ARM_EABIHF',
            'Accelerator': 'INTEL_GRAPHICS'|'MALI'|'NVIDIA'
        },
        'CompilerOptions': 'string',
        'KmsKeyId': 'string'
    },
    StoppingCondition={
        'MaxRuntimeInSeconds': 123,
        'MaxWaitTimeInSeconds': 123
    },
    Tags=[
        {
            'Key': 'string',
            'Value': 'string'
        },
    ]
)
   